In [2]:
from PIL import Image as im
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import pickle
import io
import base64
import os
import cv2
import numpy as np
import nibabel as nib
import tensorflow as tf
import matplotlib.pyplot as plt
import mysql.connector as sqlc
from nilearn import image
from nibabel import nifti1
from nibabel.viewers import OrthoSlicer3D
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add, Conv2DTranspose

In [3]:
notebook_path = os.path.abspath("preprocessing.ipynb")
scan_images = os.path.join(os.path.dirname(notebook_path), "MRI_Scans_ATLAS")
train_data = []
SIZE = 80

# get images, resize it and store it into train data list in array format
for filename in os.listdir(scan_images):
    temp = os.path.join(scan_images, filename)
    
    # read .nii file
    img = image.load_img(temp)
    width,height,queue=img.dataobj.shape
    num=1
    
    # save all the mri scans section into train data list
    for i in range(0,queue-40,1):
        img_arr=img.dataobj[:,:,i]
        img_arr = cv2.resize(img_arr, (SIZE, SIZE))
        train_data.append(img_to_array(img_arr))
        num+=1


In [4]:
# normalizing the datas into value between 0 and 1
x_train = np.reshape(train_data, (len(train_data), SIZE, SIZE, 1))
x_train = x_train.astype('float32') / 255.

In [5]:

# the code is taken from https://towardsdatascience.com/image-super-resolution-using-convolution-neural-networks-and-auto-encoders-28c9eceadf90

# split the train data into 2 section for training and validation
train_x, val_x = train_test_split(x_train, test_size=0.2)

# pixalate the images to make it low resolution image
def pixalate_image(image, scale_percent = 40):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    
    width = int(small_image.shape[1] * 100 / scale_percent)
    height = int(small_image.shape[0] * 100 / scale_percent)
    dim = (width, height)
    
    low_res_image = cv2.resize(small_image, dim, interpolation =  cv2.INTER_AREA)
    
    return low_res_image

In [6]:
# the code is taken from https://towardsdatascience.com/image-super-resolution-using-convolution-neural-networks-and-auto-encoders-28c9eceadf90

# pixalate the images in the training and validation lists
train_x_px = []
for i in range(train_x.shape[0]):
    temp = pixalate_image(train_x[i,:,:,:])
    train_x_px.append(temp)
    
train_x_px = np.array(train_x_px)

val_x_px = []
for i in range(val_x.shape[0]):
    temp = pixalate_image(val_x[i,:,:,:])
    val_x_px.append(temp)

val_x_px = np.array(val_x_px)

In [7]:

# creating the model for training

# convolution 2d is used to filter the images

# max pooling 2d is used to get the max value in the filtered images

# regularizers is used to address over-fitting
Input_img = Input(shape=(SIZE, SIZE, 1))

x1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(Input_img)
x2 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x1)
x3 = MaxPool2D(padding='same')(x2)

x4 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x3)
x5 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x4)
x6 = MaxPool2D(padding='same')(x5)

x7 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x6)
x8 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x7)
x9 = MaxPool2D(padding='same')(x8)

x10 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x9)
x11 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x10)
x12 = MaxPool2D(padding='same')(x11)

encoded = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x12)

# Up sampling is used to increase the dimension of the images to smoothen the edges
x13 = UpSampling2D()(encoded)
x14 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x13)
x15 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x14)
x16 = Add()([x11, x15])

x17 = UpSampling2D()(x16)
x18 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x17)
x19 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x18)
x20 = Add()([x8, x19])

x21 = UpSampling2D()(x20)
x22 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x21)
x23 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x22)
x24 = Add()([x5, x23])

x25 = UpSampling2D()(x24)
x26 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x25)
x27 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x26)
x28 = Add()([x2, x27])



decoded = Conv2D(3, (3, 3), padding='same',activation='relu', kernel_regularizer=regularizers.l1(10e-10))(x28)

autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 80, 80, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 80, 80, 32)   9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 40, 40, 32)   0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
# training the model
training = autoencoder.fit(train_x_px,train_x,
            epochs=1,
            validation_data=(val_x_px, val_x))
print("DONE!!")

140/176 [======================>.......] - ETA: 3s - loss: 0.0044 - accuracy: 0.0435

In [27]:
# save model
autoencoder.save('model.h5')

In [1]:
# load model
autoencoder = load_model('model.h5')
autoencoder.summary()
# predictions = autoencoder.predict(val_x_px)

NameError: name 'load_model' is not defined

In [14]:

def database_connection(): 
    """
    This function is used to access the database which will then retrieve the images and convert it from binary data
    into an image. It will then convert the image into an array and resize it to a appropriate size then normalize it
    to a value between 0 and 1 which will later be used for prediction. The predicted value will be turned from array
    into image and then the image will be converted into binary which will then be updated into the database.
    """
    connection = sqlc.connect(host='localhost',
                         database='web_app',
                         user='root',
                         password='Darrenlum1')
    
    MyCursor = connection.cursor()
    
    query = "SELECT id FROM image_uploads WHERE id IS NOT NULL AND processed_image IS NULL"
    
    MyCursor.execute(query)

    data = MyCursor.fetchall()
    for i in range(len(data)):
        temp_id = data[i][0]
        image = get_image(temp_id)

        img_val = normalize_image(image, SIZE)

        predictions = autoencoder.predict(img_val)

        convert_to_image(predictions)

        # change the image into binary
        blob_value = open("preprocessed_image.jpg", 'rb').read()

        # update the image into database
        sql = "UPDATE image_uploads SET processed_image = %s WHERE id=%s"
        args = (blob_value, temp_id)
        MyCursor.execute(sql, args)
        connection.commit()

In [19]:
def get_image(id):
    """
    This function retrieve the image from database and convert it from binary data to pil image
    """
    connection = sqlc.connect(host='localhost',
                         database='web_app',
                         user='root',
                         password='Darrenlum1')
    
    MyCursor = connection.cursor()
    
    query = "SELECT uploaded_image FROM image_uploads where id = %s"
    
    args = (id,)
    
    MyCursor.execute(query, args)
    
    data = MyCursor.fetchall()
    
    image = data[0][0]
    
    binary_data = base64.b64decode(image)

    image = im.open(io.BytesIO(binary_data))
    
    return image

In [16]:
def normalize_image(image, size):
    """
    This function resizes the image and convert it into array, then convert it into grayscale if it is not and normalizes the
    data.
    """
    
    # Resize the image
    img = image.resize((SIZE, SIZE))
    
    # convert image into array
    img_array = img_to_array(img)
    
    shp = img_array.shape
    
    # if the image is not in grayscale, change it to grayscale
    if shp[2] != 1:
        gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
        img_array = gray

    # Normalize the image
    standard_val = np.reshape(img_array, (1, SIZE, SIZE, 1))
    standard_val = standard_val.astype('float32') / 255.
    
    return standard_val

In [17]:
def convert_to_image(predictions):
    """
    This function convert the image array back to image resize it into an appropriate size and save the image as png
    """
    img_pil = array_to_img(predictions[0])
    
    img_pil = img_pil.resize((720, 720))
    img_pil.save("preprocessed_image.jpg")
    

In [61]:
database_connection()